In [1]:
from utils import get_dataframes

dataframes, effect_wheight = get_dataframes()

In [2]:
import pandas as pd
from utils import combined_damage_statistics


def damage_calculation(row: pd.Series):
    damages = row['damage']
    min_dmg, max_dmg, avg_dmg, std_dmg = combined_damage_statistics(damages)
    return min_dmg, max_dmg, avg_dmg, std_dmg


weapons_df = dataframes["weapons"]
weapons_df[["min_dmg", "max_dmg", "avg_dmg", "std_dmg"]] = weapons_df.apply(
    damage_calculation,
    axis=1,
    result_type="expand"
)

# weapons_df.sort_values(by=["weapon_type", "avg_dmg"])[["name", "weapon_type", "avg_dmg"]]
weapons_df

,name,weapon_type,prerequisites,range_,range_far,damage,abilities,min_dmg,max_dmg,avg_dmg,std_dmg
0,Banditen-Wurfbeilpaar,Äxte,"[[15, Str], [13, Dex]]",5,40,"[[1d8+4, slashing], [1d8+6, slashing]]","[Duo-Waffe, Zweihändig]",12.0,26.0,19.0,3.240370
1,Blutdolch der Sehnsüchte,Dolche,"[[15, Dex]]",5,30,"[[1d8, piercing], [2d8, acid]]",[Doppelter Angriff],3.0,24.0,13.5,3.968627
2,Blutstahlritter-Großschwert,Großschwerter,"[[16, Str], [11, Dex]]",10,0,"[[3d12, slashing], [2d4, thunder]]",[Zweihändig],5.0,44.0,24.5,6.184658
3,Bogen des Morgenrots,Kurzbögen,"[[16, Dex]]",55,80,"[[3d8, piercing], [2d10, fire]]","[Strahl des Morgenrots, Kraft des Morgenrots]",5.0,44.0,24.5,5.678908
4,Dolch der Täuschung,Dolche,"[[16, Dex]]",5,30,"[[3d6, piercing], [4d6, psychic]]",[],7.0,42.0,24.5,4.518481
5,Doppelschussbogen,Kurzbögen,"[[15, Dex]]",60,90,"[[2d8+2, piercing]]","[Zweihändig, Doppelschuss]",4.0,18.0,11.0,3.240370
6,Dornenbogen,Kurzbögen,"[[14, Dex]]",50,80,"[[2d12+4, piercing], [3d4, piercing]]","[Zweihändig, Dornenrankenexplosion]",9.0,40.0,24.5,5.251984
7,Drachenschlächtergroßaxt,Großäxte,"[[16, Str]]",5,0,"[[2d12, slashing], [4d4, lightning]]","[Zweihändig, Drachenblitz]",6.0,40.0,23.0,5.369668
8,Einfacher Dolch,Dolche,"[[10, Dex]]",5,30,"[[1d4, piercing]]",[Doppelter Angriff],1.0,4.0,2.5,1.118034
9,Elfisches Silberjagdmesser,Dolche,"[[13, Dex]]",5,35,"[[1d4, piercing], [1d8, psychic]]","[Lautloser Wanderer, Doppelter Angriff]",2.0,12.0,7.0,2.549510


In [3]:
import re
import pandas as pd

# Load weapon and ability data
weapons_df = dataframes["weapons"]
weapon_abilities_df = dataframes["weapon_abilities"]


### STEP 1: Compute Damage Statistics ###
def damage_calculation(row: pd.Series):
    """
    Computes min, max, avg, and std deviation for weapon damage.
    """
    damages = row['damage']
    min_dmg, max_dmg, avg_dmg, std_dmg = combined_damage_statistics(damages)
    return min_dmg, max_dmg, avg_dmg, std_dmg


# Compute damage statistics and store them as new columns
weapons_df[["min_dmg", "max_dmg", "avg_dmg", "std_dmg"]] = weapons_df.apply(
    damage_calculation,
    axis=1,
    result_type="expand"
)

### STEP 2: Define Patterns for Ability Extraction ###
effect_patterns = {
    "extra_damage":
        {"pattern": r"(zusätzlichen Schaden|Schaden von \d+W\d+|Schadenswurf)", "value": 3, "damage": r"(\d+W\d+)"},
    "double_attack":
        {"pattern": r"(doppelt angegriffen|zwei Treffer|zwei Geschosse)", "value": 4},
    "paralysed":
        {"pattern": r"(paralysiert|gelähmt|Bewegungsunfähig)", "value": 5, "duration": r"(\d+)\s*Runden"},
    "bonus_action":
        {"pattern": r"(Bonusaktion|zusätzliche Aktion)", "value": 2},
    "range_bonus":
        {"pattern": r"(Reichweite\s*\+\d+|weiter schießen|längere Reichweite)", "value": 2, "bonus": r"(\d+)ft"},
    "area_effect":
        {"pattern": r"(Explosion|rotierender Angriff|mehrere Ziele)", "value": 3},
    "exhausted":
        {"pattern": r"(erschöpft|Ermüdung|Ausdauerverlust|Kraftverlust)", "value": 5},
    "petrified":
        {"pattern": r"(versteinert|zu Stein verwandelt)", "value": 5},
    "bleeding":
        {"pattern": r"(Blutung|blutet|schwere Wunde|ausbluten)", "value": 5, "damage": r"(\d+W\d+)"},
    "knockback":
        {"pattern": r"(\d+ft. zurück|zurückgestoßen|zurückgedrängt)", "value": 3, "distance": r"(\d+)ft"},
    "disadvantage_(hit)":
        {"pattern": r"(Nachteil auf Angriff|Angriffe haben Nachteil)", "value": 4},
    "disadvantage_(dmg)":
        {"pattern": r"(Nachteil auf Schaden|Schaden verringert)", "value": 4},
    "stunned":
        {"pattern": r"(betäubt|benommen|Handlungsunfähig)", "value": 4, "duration": r"(\d+)\s*Runden"},
    "poisoned":
        {"pattern": r"(vergiftet|Giftwirkung|Giftresistenz erforderlich)", "value": 3, "duration": r"(\d+)\s*Runden"},
    "grappled":
        {"pattern": r"(gepackt|umklammert|festgehalten)", "value": 3},
    "prone":
        {"pattern": r"(zu Boden geworfen|liegt am Boden|hingeworfen)", "value": 3},
    "double_damage":
        {"pattern": r"(doppelter Schaden|Schaden x2|Schadensmultiplikator)", "value": 4},
    "heal":
        {"pattern": r"(heilen|stellt \d+ Lebenspunkte wieder her|Heilwirkung)", "value": 2, "heal_amount": r"(\d+) HP"},
    "leap":
        {"pattern": r"(Sprung|springt über|kann weiter springen)", "value": 2, "bonus": r"(\d+)ft"},
    "frightened":
        {"pattern": r"(verängstigt|erschrocken|Fluchtreflex|Furcht)", "value": 3},
    "blinded":
        {"pattern": r"(geblendet|kann nicht sehen|blind)", "value": 3},
    "pull":
        {"pattern": r"(zieht das Ziel heran|heranziehen|zieht Gegner näher)", "value": 2, "distance": r"(\d+)ft"},
    "increased_movement":
        {"pattern": r"(Bewegungsgeschwindigkeit erhöht|schneller bewegen|mehr Reichweite)", "value": 3,
         "bonus": r"(\d+)ft"},
    "escape":
        {"pattern": r"(kann entkommen|leicht flüchten|Fluchtmöglichkeit)", "value": 3},
    "invisible":
        {"pattern": r"(unsichtbar|nicht sichtbar|getarnt)", "value": 4, "duration": r"(\d+)\s*Runden"},
    "restrained":
        {"pattern": r"(bewegungsunfähig gemacht|gefesselt|unbeweglich)", "value": 4},
    "resistance_(all)":
        {"pattern": r"(Resistenz gegen alle Schäden|reduzierter Schaden|Schutz vor Schaden)", "value": 4},
    "difficult_terrain":
        {"pattern": r"(erschwertes Gelände|langsamer bewegen|schwierige Fortbewegung)", "value": 2},
}


### STEP 3: Extract Effects from Weapon Abilities ###
def extract_effects(description):
    """
    Extracts ability effects and stores them in a structured dictionary.
    """
    effects = {}
    for effect, data in effect_patterns.items():
        match = re.search(data["pattern"], description, re.IGNORECASE) if isinstance(description, str) else None
        if match:
            effect_data = {"value": data["value"]}
            for attr, regex in data.items():
                if attr != "pattern" and attr != "value":
                    extra_match = re.search(regex, description)
                    if extra_match:
                        effect_data[attr] = int(extra_match.group(1)) if extra_match.group(
                            1).isdigit() else extra_match.group(1)

            effects[effect] = effect_data
    return effects


def apply_effect_impact(row: pd.Series):
    effect = row["effects"]
    if isinstance(effect, dict) and effect:
        effect_name = list(effect.keys())[0]
        effect_params = effect[effect_name]
        effect_value = effect_params.get('value', 0)

        if 'effect_value' not in row or pd.isna(row['effect_value']):
            row['effect_value'] = 0

        match effect_name:
            case 'double_attack':
                row['effect_value'] += row['avg_dmg']
            case 'heal':
                row['effect_value'] += effect_value
            case 'knockback' | 'pull' | 'invisibility':
                if 'distance' in effect_params:
                    distance = effect_params.get('distance', 0)
                    row['effect_value'] += effect_value * (distance / 5) ** (1 / 2)
                if 'duration' in effect_params:
                    duration = effect_params.get('duration', 0)
                    row['effect_value'] += effect_value * duration
                row['effect_value'] += effect_value
            case _:
                row['effect_value'] += effect_value
    else:
        row['effect_value'] = row.get('effect_value', 0)
    return row


# Apply function to extract structured effects
weapon_abilities_df["effects"] = weapon_abilities_df["description"].apply(
    lambda x: extract_effects(x) if isinstance(x, str) else {})

weapon_abilities_df.rename(columns={"name": "effect_name"}, inplace=True)

weapons_exploded = weapons_df.explode("abilities")
merged_df = weapons_exploded.merge(
    weapon_abilities_df[["effect_name", "effects"]],
    left_on="abilities", right_on="effect_name", how="left"
).drop("effect_name", axis=1)

merged_df = merged_df.apply(apply_effect_impact, axis=1)

merged_agg_df = merged_df.groupby('name').agg({
    'abilities': lambda x: ', '.join(x.dropna().unique()),  # Combine unique abilities
    'avg_dmg': 'mean',  # Average damage (can also use sum if needed)
    'min_dmg': 'min',
    'max_dmg': 'max',
    'effect_value': 'sum',  # Sum of effect values
    'damage': 'first',  # Keep the first damage value (assuming same for all)
    'prerequisites': 'first',  # Keep prerequisites (assuming same)
    'range_': 'first',
    'range_far': 'first',
    'std_dmg': 'mean',
    'weapon_type': 'first'  # Keep weapon type (assuming same)
}).reset_index()

merged_agg_df["strength_metric"] = round((
        merged_agg_df["avg_dmg"] +
        merged_agg_df["range_"] / 5 * 0.2 +
        (merged_agg_df["range_far"] / 5) ** (1 / 2) * 0.2 +
        merged_agg_df["effect_value"]
), 2)

analysed_weapon_df = merged_agg_df.copy()

analysed_weapon_df

,name,abilities,avg_dmg,min_dmg,max_dmg,effect_value,damage,prerequisites,range_,range_far,std_dmg,weapon_type,strength_metric
0,Banditen-Wurfbeilpaar,"Duo-Waffe, Zweihändig",19.0,12.0,26.0,19.0,"[[1d8+4, slashing], [1d8+6, slashing]]","[[15, Str], [13, Dex]]",5,40,3.240370,Äxte,38.77
1,Blutdolch der Sehnsüchte,Doppelter Angriff,13.5,3.0,24.0,13.5,"[[1d8, piercing], [2d8, acid]]","[[15, Dex]]",5,30,3.968627,Dolche,27.69
2,Blutstahlritter-Großschwert,Zweihändig,24.5,5.0,44.0,0.0,"[[3d12, slashing], [2d4, thunder]]","[[16, Str], [11, Dex]]",10,0,6.184658,Großschwerter,24.90
3,Bogen des Morgenrots,"Strahl des Morgenrots, Kraft des Morgenrots",24.5,5.0,44.0,0.0,"[[3d8, piercing], [2d10, fire]]","[[16, Dex]]",55,80,5.678908,Kurzbögen,27.50
4,Dolch der Täuschung,,24.5,7.0,42.0,0.0,"[[3d6, piercing], [4d6, psychic]]","[[16, Dex]]",5,30,4.518481,Dolche,25.19
5,Doppelschussbogen,"Zweihändig, Doppelschuss",11.0,4.0,18.0,11.0,"[[2d8+2, piercing]]","[[15, Dex]]",60,90,3.240370,Kurzbögen,25.25
6,Dornenbogen,"Zweihändig, Dornenrankenexplosion",24.5,9.0,40.0,0.0,"[[2d12+4, piercing], [3d4, piercing]]","[[14, Dex]]",50,80,5.251984,Kurzbögen,27.30
7,Drachenschlächtergroßaxt,"Zweihändig, Drachenblitz",23.0,6.0,40.0,0.0,"[[2d12, slashing], [4d4, lightning]]","[[16, Str]]",5,0,5.369668,Großäxte,23.20
8,Einfacher Dolch,Doppelter Angriff,2.5,1.0,4.0,2.5,"[[1d4, piercing]]","[[10, Dex]]",5,30,1.118034,Dolche,5.69
9,Elfisches Silberjagdmesser,"Lautloser Wanderer, Doppelter Angriff",7.0,2.0,12.0,7.0,"[[1d4, piercing], [1d8, psychic]]","[[13, Dex]]",5,35,2.549510,Dolche,14.73


In [4]:
def find_strongest_weapon(weapons_df: pd.DataFrame, user_attributes: list[tuple[str, int]],
                          prio_weapon_type: list[str]):
    user_attr_dict = {attr[0].lower(): attr[1] for attr in user_attributes}

    def meets_prerequisites(row):
        prerequisites = row['prerequisites']
        for value, attribute in prerequisites:
            if user_attr_dict.get(attribute.lower(), 0) < int(value):
                return False
        return True

    qualified_weapons = weapons_df[weapons_df.apply(meets_prerequisites, axis=1)]

    if prio_weapon_type:
        prio_weapon_type = [wtype.strip().lower() for wtype in prio_weapon_type]
        qualified_weapons = qualified_weapons.copy()  # Avoid chained assignment issues
        qualified_weapons.loc[:, 'weapon_type'] = qualified_weapons['weapon_type'].str.strip().str.lower()
        qualified_weapons = qualified_weapons[qualified_weapons['weapon_type'].isin(prio_weapon_type)]

    if not qualified_weapons.empty:
        best_weapon = qualified_weapons.loc[qualified_weapons['strength_metric'].idxmax()]
        return best_weapon[['name', 'weapon_type', 'strength_metric']].to_dict()
    else:
        return "No weapons match your qualifications."


user_stats = [["str", 14], ["dex", 17]]
prio_weapon_types = ["Dolche", "Stichschwerter"]
best_weapon = find_strongest_weapon(analysed_weapon_df, user_stats, prio_weapon_types)
best_weapon


{'name': 'Blutdolch der Sehnsüchte',
 'weapon_type': 'dolche',
 'strength_metric': 27.69}